*Install Libraries*

---



In [ ]:
import pandas as pd

In [ ]:
!pip install openai
from openai import OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


***Open API Key***

In [ ]:
import os
import ast
import numpy as np
import pdb
api_key = '' # Your Api Key
client = OpenAI(api_key=api_key)

*Sample Test Run, with simple Prompt*

In [ ]:
question = "What is the capital of France?"
question

'What is the capital of France?'

Sending Prompt to Language Model using Open AI API

In [ ]:
response = client.chat.completions.create(
    model ="gpt-3.5-turbo", messages= [{"role": "user", "content": question}]
)
response

ChatCompletion(id='chatcmpl-9gviJU5ErErXgIQuP3Yk2MqsJSb0u', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None))], created=1720018655, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=14, total_tokens=21))

In [ ]:
response.choices[0].message.content

'The capital of France is Paris.'

#Setting up Retrieval Augmented Generation
**Web Scrapping**

*Scrapping Local Weather Website to use it as Retrival Model using basic Webscrapping Techniques*

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

URL = "https://forecast.weather.gov/MapClick.php?lat=37.7771&lon=-122.4196#.Y-62qOzMJZM"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")


In [ ]:
# results = soup.find(id="current_conditions-summary")
# if results:
#     job_elements = results.find_all(class_="myforecast-current-lrg")

#     for job_element in job_elements:
#         period_element = job_element.find("p", class_="myforecast-current-lrg")
#         temp_element = job_element.find("p", class_="myforecast-current")
#         #short_element = job_element.find(class_="short-desc")
#         print(period_element.text)
#         print(temp_element.text)
#         #print(short_element.text)
#         print()
# else:
#     print("Could not find element with id 'seven-day-forecast'")


In [ ]:
def get_current_conditions(soup):
    current_conditions = {}
    current_conditions_element = soup.find(id="current_conditions-summary")
    if current_conditions_element:
        temp_element = current_conditions_element.find("p", class_="myforecast-current-lrg")
        condition_element = current_conditions_element.find("p", class_="myforecast-current")
        current_conditions["temperature"] = temp_element.text.strip() if temp_element else "N/A"
        current_conditions["condition"] = condition_element.text.strip() if condition_element else "N/A"
    return current_conditions

def get_extended_forecast(soup):
    forecast = []
    seven_day_forecast = soup.find(id="seven-day-forecast")
    if seven_day_forecast:
        forecast_items = seven_day_forecast.find_all("li", class_="forecast-tombstone")
        for item in forecast_items:
            period = item.find("p", class_="period-name").text.strip()
            short_desc = item.find("p", class_="short-desc").text.strip()
            #temp = item.find("p", class_="temp").text.strip()
            forecast.append({"period": period, "short_desc": short_desc})
    return forecast

def get_severe_weather_alerts(soup):
    alerts = []
    alert_element = soup.find("div", class_="headline-bar")
    if alert_element:
        alert_text = alert_element.find("span", attrs={"role": "alert"})
        if alert_text:
            alerts.append(alert_text.text.strip())
    return alerts

response = requests.get('https://forecast.weather.gov/MapClick.php?lat=37.7771&lon=-122.4196#.Y-62qOzMJZM')

soup = BeautifulSoup(response.content, "html.parser")

current_conditions = get_current_conditions(soup)
extended_forecast = get_extended_forecast(soup)
severe_weather_alerts = get_severe_weather_alerts(soup)

print("Current Conditions:")
for key, value in current_conditions.items():
    print(f"{key}: {value}")

print("\nExtended Forecast:")
for day in extended_forecast:
    print(f"{day['period']}: {day['short_desc']}")

print("\nSevere Weather Alerts:")
for alert in severe_weather_alerts:
    print(alert)



Current Conditions:
temperature: 59°F
condition: NA

Extended Forecast:
NOW: Multiple hazards in effect: Click HERE for Details
Today: Sunny
Tonight: Mostly Clear
Independence Day: Sunny
Thursday Night: Partly Cloudy
Friday: Sunny
Friday Night: Mostly Clear
Saturday: Sunny
Saturday Night: Mostly Clear

Severe Weather Alerts:


In [ ]:
current_conditions

{'temperature': '59°F', 'condition': 'NA'}

In [ ]:
severe_weather_alerts

[]

*Importing Scrapped Information into Dataframe*

In [ ]:
df = pd.DataFrame(extended_forecast)
df

,period,short_desc
0,NOW: Multiple hazards in effect,Click HERE for Details
1,Today,Sunny
2,Tonight,Mostly Clear
3,Independence Day,Sunny
4,Thursday Night,Partly Cloudy
5,Friday,Sunny
6,Friday Night,Mostly Clear
7,Saturday,Sunny
8,Saturday Night,Mostly Clear


In [ ]:
df1= pd.DataFrame(current_conditions,index=[0])
df1

,temperature,condition
0,59°F,NA


In [ ]:
df2=df.join(df1)

In [ ]:
df2

,period,short_desc,temperature,condition
0,NOW: Multiple hazards in effect,Click HERE for Details,59°F,NA
1,Today,Sunny,NaN,NaN
2,Tonight,Mostly Clear,NaN,NaN
3,Independence Day,Sunny,NaN,NaN
4,Thursday Night,Partly Cloudy,NaN,NaN
5,Friday,Sunny,NaN,NaN
6,Friday Night,Mostly Clear,NaN,NaN
7,Saturday,Sunny,NaN,NaN
8,Saturday Night,Mostly Clear,NaN,NaN


*Scrapping further Informations From the Website*

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_detailed_forecast(soup):
    detailed_forecast = []
    detailed_forecast_table = soup.find("div", id="detailed-forecast-body")
    if detailed_forecast_table:
        forecast_rows = detailed_forecast_table.find_all("div", class_="row")
        for row in forecast_rows:
            period = row.find("div", class_="forecast-label").text.strip()
            short_desc = row.find("div", class_="forecast-text").text.strip()
            detailed_forecast.append({"period": period, "short_desc": short_desc})
    return detailed_forecast

response = requests.get('https://forecast.weather.gov/MapClick.php?lat=37.7771&lon=-122.4196#.Y-62qOzMJZM')
#response.raise_for_status()

soup = BeautifulSoup(response.content, "html.parser")

detailed_forecast = get_detailed_forecast(soup)

print("\nDetailed Forecast:")
for entry in detailed_forecast:
    print(f"{entry['period']}: {entry['short_desc']}")


Detailed Forecast:
Today: Sunny, with a high near 84. Light west southwest wind becoming west 6 to 11 mph in the afternoon.
Tonight: Mostly clear, with a low around 57. West southwest wind 5 to 8 mph.
Independence Day: Sunny, with a high near 83. Light west southwest wind increasing to 5 to 10 mph in the morning.
Thursday Night: Partly cloudy, with a low around 57. West southwest wind 3 to 8 mph.
Friday: Sunny, with a high near 83. Light west southwest wind becoming west 5 to 10 mph in the afternoon.
Friday Night: Mostly clear, with a low around 58.
Saturday: Sunny, with a high near 83.
Saturday Night: Mostly clear, with a low around 58.
Sunday: Sunny, with a high near 81.
Sunday Night: Mostly clear, with a low around 57.
Monday: Sunny, with a high near 78.
Monday Night: Partly cloudy, with a low around 56.
Tuesday: Sunny, with a high near 75.


In [ ]:
detailed_forecast

[{'period': 'Today',
  'short_desc': 'Sunny, with a high near 84. Light west southwest wind becoming west 6 to 11 mph in the afternoon.'},
 {'period': 'Tonight',
  'short_desc': 'Mostly clear, with a low around 57. West southwest wind 5 to 8 mph.'},
 {'period': 'Independence Day',
  'short_desc': 'Sunny, with a high near 83. Light west southwest wind increasing to 5 to 10 mph in the morning.'},
 {'period': 'Thursday Night',
  'short_desc': 'Partly cloudy, with a low around 57. West southwest wind 3 to 8 mph.'},
 {'period': 'Friday',
  'short_desc': 'Sunny, with a high near 83. Light west southwest wind becoming west 5 to 10 mph in the afternoon.'},
 {'period': 'Friday Night',
  'short_desc': 'Mostly clear, with a low around 58.'},
 {'period': 'Saturday', 'short_desc': 'Sunny, with a high near 83.'},
 {'period': 'Saturday Night',
  'short_desc': 'Mostly clear, with a low around 58.'},
 {'period': 'Sunday', 'short_desc': 'Sunny, with a high near 81.'},
 {'period': 'Sunday Night',
  'shor

*Importing into DataFrame*

In [ ]:
df3 = pd.DataFrame(detailed_forecast)
df3

,period,short_desc
0,Today,"Sunny, with a high near 84. Light west southwe..."
1,Tonight,"Mostly clear, with a low around 57. West south..."
2,Independence Day,"Sunny, with a high near 83. Light west southwe..."
3,Thursday Night,"Partly cloudy, with a low around 57. West sout..."
4,Friday,"Sunny, with a high near 83. Light west southwe..."
5,Friday Night,"Mostly clear, with a low around 58."
6,Saturday,"Sunny, with a high near 83."
7,Saturday Night,"Mostly clear, with a low around 58."
8,Sunday,"Sunny, with a high near 81."
9,Sunday Night,"Mostly clear, with a low around 57."


In [ ]:
df4 = pd.concat([df,df3], ignore_index=True)

In [ ]:
df4

,period,short_desc
0,NOW: Multiple hazards in effect,Click HERE for Details
1,Today,Sunny
2,Tonight,Mostly Clear
3,Independence Day,Sunny
4,Thursday Night,Partly Cloudy
5,Friday,Sunny
6,Friday Night,Mostly Clear
7,Saturday,Sunny
8,Saturday Night,Mostly Clear
9,Today,"Sunny, with a high near 84. Light west southwe..."


In [ ]:
df4=df4.iloc[1:]

In [ ]:
df4["Period_shortdescp"] = df4["period"] + ":" + df4["short_desc"]

<ipython-input-53-a2b8199f1033>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4["Period_shortdescp"] = df4["period"] + ":" + df4["short_desc"]


In [ ]:
df4

,period,short_desc,Period_shortdescp
1,Today,Sunny,Today:Sunny
2,Tonight,Mostly Clear,Tonight:Mostly Clear
3,Independence Day,Sunny,Independence Day:Sunny
4,Thursday Night,Partly Cloudy,Thursday Night:Partly Cloudy
5,Friday,Sunny,Friday:Sunny
6,Friday Night,Mostly Clear,Friday Night:Mostly Clear
7,Saturday,Sunny,Saturday:Sunny
8,Saturday Night,Mostly Clear,Saturday Night:Mostly Clear
9,Today,"Sunny, with a high near 84. Light west southwe...","Today:Sunny, with a high near 84. Light west s..."
10,Tonight,"Mostly clear, with a low around 57. West south...","Tonight:Mostly clear, with a low around 57. We..."


##Embedding Retrived Information
*Creating Basic Embedding Vector for retrived Information from the website*

In [ ]:
def get_embeddings(text,model="text-embedding-3-small"):
  text =text.replace("\n","")
  return client.embeddings.create(input=[text],model=model).data[0].embedding

In [ ]:
get_embeddings(df4['Period_shortdescp'].iloc[0])

[0.002805584343150258,
 -0.05045521631836891,
 -0.013629897497594357,
 -0.00312594510614872,
 0.041679270565509796,
 0.023182475939393044,
 0.003841094207018614,
 0.0008882732363417745,
 0.004954267293214798,
 0.010937572456896305,
 -0.0068214209750294685,
 0.029304927214980125,
 -0.008678866550326347,
 -0.011300000362098217,
 -0.03634639456868172,
 0.03318808972835541,
 -0.04520000144839287,
 0.024502750486135483,
 -0.005779439117759466,
 -0.011248225346207619,
 0.05488201975822449,
 -0.01860034465789795,
 0.008038144558668137,
 -0.008970103226602077,
 -2.3606135073350742e-06,
 0.04457869753241539,
 -0.028787171468138695,
 0.029822682961821556,
 0.06332142651081085,
 -0.017746049910783768,
 0.023752005770802498,
 -0.03168660029768944,
 0.00035595649387687445,
 -0.01284032128751278,
 0.022004583850502968,
 0.027259794995188713,
 0.017875486984848976,
 0.02601718343794346,
 -0.04558831825852394,
 -0.003789318725466728,
 0.028968386352062225,
 -0.04134272783994675,
 -0.005743843503296375

In [ ]:
df4['embedding'] = df4['Period_shortdescp'].apply(get_embeddings)

<ipython-input-58-42c4ed37de7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['embedding'] = df4['Period_shortdescp'].apply(get_embeddings)


**Creating embeddings for the user Prompt**

In [ ]:
question = "How is the Climate on Friday?"
question

'How is the Climate on Friday?'

In [ ]:
question_embedding =get_embeddings(question)
question, question_embedding[0:10],"Cont..."

('How is the Climate on Friday?',
 [-0.01158934086561203,
  -0.019001364707946777,
  -0.003058392321690917,
  -0.01255907490849495,
  -0.0027939192950725555,
  0.015922628343105316,
  -0.00954815186560154,
  -0.022459857165813446,
  -0.018947113305330276,
  0.006449071224778891],
 'Cont...')

*Calculting the similarity between Question embedding and Retrived infromation embeddings*

In [ ]:
def fn(data_embedding ):
  return np.dot(data_embedding,question_embedding)
df4['similarity'] = df4['embedding'].apply(fn)

<ipython-input-61-110c84d5bbe1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['similarity'] = df4['embedding'].apply(fn)


Higher the similarity value which means the Question and information are in closest distance in the vector Graph.

In [ ]:
df4.sort_values(by='similarity',ascending=False, inplace=True)
df4

<ipython-input-64-a17956d150ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.sort_values(by='similarity',ascending=False, inplace=True)


,period,short_desc,Period_shortdescp,embedding,similarity
5,Friday,Sunny,Friday:Sunny,"[0.022825732827186584, -0.06095673516392708, -...",0.539573
13,Friday,"Sunny, with a high near 83. Light west southwe...","Friday:Sunny, with a high near 83. Light west ...","[0.00034259987296536565, -0.013948917388916016...",0.521462
4,Thursday Night,Partly Cloudy,Thursday Night:Partly Cloudy,"[0.002076831879094243, -0.020273758098483086, ...",0.468700
14,Friday Night,"Mostly clear, with a low around 58.","Friday Night:Mostly clear, with a low around 58.","[-0.02411586232483387, 0.009336842224001884, -...",0.462949
1,Today,Sunny,Today:Sunny,"[0.002805584343150258, -0.05045521631836891, -...",0.457633
6,Friday Night,Mostly Clear,Friday Night:Mostly Clear,"[-0.0025204308331012726, -0.004422837868332863...",0.457567
15,Saturday,"Sunny, with a high near 83.","Saturday:Sunny, with a high near 83.","[0.022292839363217354, -0.006009022705256939, ...",0.436298
16,Saturday Night,"Mostly clear, with a low around 58.","Saturday Night:Mostly clear, with a low around...","[-0.016677897423505783, 0.01363922655582428, -...",0.435496
12,Thursday Night,"Partly cloudy, with a low around 57. West sout...","Thursday Night:Partly cloudy, with a low aroun...","[-0.029411913827061653, -0.001791545539163053,...",0.430755
19,Monday,"Sunny, with a high near 78.","Monday:Sunny, with a high near 78.","[-0.014277216047048569, 0.0028073338326066732,...",0.430655


# Prompt engineering- Augumentation
Augumenting the closes information into the Question Prompt.

In [ ]:
context = df4['Period_shortdescp'].iloc[0]+ " "+ df4['Period_shortdescp'].iloc[1] + " " + df4['Period_shortdescp'].iloc[2] + " " + df4['Period_shortdescp'].iloc[3] + " " + df4['Period_shortdescp'].iloc[4]
context

'Friday:Sunny Friday:Sunny, with a high near 83. Light west southwest wind becoming west 5 to 10 mph in the afternoon. Thursday Night:Partly Cloudy Friday Night:Mostly clear, with a low around 58. Today:Sunny'

Final Response With Retrived Information

In [ ]:
response = client.chat.completions.create(
    model ="gpt-3.5-turbo", messages= [{"role": "system", "content": "Your an Weather Assistant, who is helping people about quick wether update"},
                                       {"role": "user", "content": question},
                                       {"role":"assistant","content":f"Use the information from Local weather website to answer the user Question:{context}.Please stick to the context when answering the user question."}]
)
response.choices[0].message.content

'On Friday, the weather is expected to be sunny with a high near 83. There will be a light west southwest wind that will become west 5 to 10 mph in the afternoon. The night will be mostly clear with a low around 58.'